# Découverte des données OpenFoodFacts

In [1]:
# On importe les librairies dont on a besoin
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from zipfile import ZipFile
from __future__ import print_function
import ipywidgets as widgets
from IPython.display import display, clear_output
from sklearn import decomposition
from sklearn import preprocessing
%matplotlib widget

In [2]:
# Fonction de projection des valeurs sur le cercle des correlations
def display_factorial_planes(X_projected, n_comp, pca, axis_ranks, labels=None, alpha=1, illustrative_var=None, colors=None):
    for d1,d2 in axis_ranks:
        if d2 < n_comp:
 
            # initialisation de la figure       
            #fig = plt.figure()
            fig, ax = plt.subplots(1)
        
            # affichage des points
            if illustrative_var is None:
                #plt.scatter(X_projected[:, d1], X_projected[:, d2], alpha=alpha, c=colors, s=100)
                ax.scatter(X_projected[:, d1], X_projected[:, d2], alpha=alpha, c=colors, s=100)
            else:
                illustrative_var = np.array(illustrative_var)
                for value in np.unique(illustrative_var):
                    selected = np.where(illustrative_var == value)
                    if colors != None : 
                        color = colors[selected]
                    else :
                        color = 'blue'
                    #plt.scatter(X_projected[selected, d1], X_projected[selected, d2], alpha=alpha, label=value, c=color, s=100)
                    ax.scatter(X_projected[selected, d1], X_projected[selected, d2], alpha=alpha, label=value, c=color, s=100)
                plt.legend()

            # affichage des labels des points
            if labels is not None:
                for i,(x,y) in enumerate(X_projected[:,[d1,d2]]):
                    plt.text(x, y, labels[i],
                              fontsize='14', ha='center',va='center') 
                
            # détermination des limites du graphique
            boundary = np.max(np.abs(X_projected[:, [d1,d2]])) * 1.3
            plt.xlim([-boundary,boundary])
            plt.ylim([-boundary,boundary])
        
            # affichage des lignes horizontales et verticales
            plt.plot([-100, 100], [0, 0], color='grey', ls='--')
            plt.plot([0, 0], [-100, 100], color='grey', ls='--')

            # nom des axes, avec le pourcentage d'inertie expliqué            
            plt.xlabel('Richesse nutritive du produit')
            plt.ylabel('Transformation industrielle')

            plt.title('Richesse nutritionnelle / transformation industrielle.')          
            
            plt.show(block=False)

In [3]:
# Fonction qui renvoie une liste de couleur à partir du dataFrame, de la colonne à traiter
# et d'un dictionnaire contenant les valeurs et les couleurs à appliquer
#Ex: Utilisé pour visualiser le nutriscore par couleur
def pltcolor(unDataFrame, uneVariable, unDict):
    cols=[]
    for uneCategorie in unDataFrame[uneVariable]:
        if uneVariable == 'nutrition-score-fr_100g':
            cols.append(unDict[getNutriscore(uneCategorie)])
        else:
            cols.append(unDict[uneCategorie])
    return cols

In [4]:
# on charge le fichier dans un dataFrame
data = pd.read_csv("fr.openfoodfacts.org.products.nettoye.knn.csv",decimal=".", low_memory=False)

In [5]:
# dictionnaire lettre/couleur pour le nutriscore
nutri_color_dict = {}
nutri_color_dict['A'] = 'darkgreen'
nutri_color_dict['B'] = 'limegreen'
nutri_color_dict['C'] = 'yellow'
nutri_color_dict['D'] = 'orange'
nutri_color_dict['E'] = 'red'

In [6]:
# fonction qui renvoie le nutriscore sous forme de lettre à partir de la note chiffrée

def getNutriscore(valeur):
    if valeur >= -15  and valeur <=-2:
        return "A"
    elif valeur >= -1 and valeur <= 3:
        return "B"
    elif valeur >= 4 and valeur <= 11:
        return "C"
    elif valeur >= 12 and valeur <= 16:
        return "D"
    elif valeur >= 17 and valeur <= 40:
        return "E"

## Répartition du nutriscore en fonction du pays et de la catégorie

In [7]:
def graph_nutriscore_categorie(*args):
    if dropdown_categ.value == None :
        update_cat(dropdown_pays)
    data_filtre = data[data['countries_fr'] == dropdown_pays.value]

    with nutr_cat_output:
        nutr_cat_output.clear_output(True)
        fig = plt.figure(figsize=(8,8))
        histo_cat_nutri = sns.countplot(y="main_category_fr", \
                                        hue="nutrition-letter-fr_100g",\
                                        hue_order=["A", "B","C","D","E"],\
                                        data=data_filtre[data_filtre['main_category_fr']==dropdown_categ.value], \
                                        palette=nutri_color_dict)
        histo_cat_nutri.set_ylabel('Catégories', fontsize = 15)
        histo_cat_nutri.set_xlabel('Nombre de produits', fontsize = 15)
        histo_cat_nutri.set_title('Répartition du nutriscore par catégorie', fontsize = 20)        
        plt.yticks(rotation=90)
        plt.show()
        
dropdown_pays = widgets.Dropdown(
    value='France',
    options = data['countries_fr'].value_counts().index.sort_values(),
    description = 'Liste des pays:',
    style = {'description_width': 'initial'}
    )

dropdown_categ = widgets.Dropdown(
    description = 'Liste des catégories:',
    style = {'description_width': 'initial'}
    )

def update_cat(*args):    
    data_pays = data[data['countries_fr'] == dropdown_pays.value]
    dropdown_categ.options = data_pays['main_category_fr'].value_counts().index.sort_values()
    dropdown_categ.value = dropdown_categ.options[0]
    graph_nutriscore_categorie(dropdown_categ,dropdown_pays)

dropdown_pays.observe(update_cat, 'value')
dropdown_categ.observe(graph_nutriscore_categorie, 'value')

nutr_cat_output = widgets.Output()

widgets.interact(graph_nutriscore_categorie, Pays = dropdown_pays, Catégorie = dropdown_categ)
widgets.VBox(
    [
        dropdown_pays,
        dropdown_categ, 
        nutr_cat_output
    ]
)





interactive(children=(Output(),), _dom_classes=('widget-interact',))

## Répartition du nutriscore en fonction du pays et des marques

In [8]:
def graph_nutriscore_marque(*args):
    if dropdown_marque.value == None :
        update_marque(dropdown_pays_mq)
    data_filtre = data[data['countries_fr'] == dropdown_pays_mq.value]
    
    with nutr_marque_output:
        nutr_marque_output.clear_output(True)
        plt.figure(figsize=(8, 8))
        histo_marque_nutri = sns.countplot(y="brands", hue="nutrition-letter-fr_100g",\
                                           hue_order=["A", "B","C","D","E"], \
                                           data=data_filtre[data_filtre['brands']==dropdown_marque.value], \
                                           palette=nutri_color_dict)
        histo_marque_nutri.set_ylabel('Marques', fontsize = 15)
        histo_marque_nutri.set_xlabel('Nombre de produits', fontsize = 15)
        histo_marque_nutri.set_title('Répartition du nutriscore par marque', fontsize = 20)
        plt.yticks(rotation=90)
        plt.show()

dropdown_pays_mq = widgets.Dropdown(
    value='France',
    options = data['countries_fr'].value_counts().index.sort_values(),
    description = 'Liste des pays:',
    style = {'description_width': 'initial'}
    )

dropdown_marque = widgets.Dropdown(
    description = 'Liste des marques:',
    style = {'description_width': 'initial'}
    )

def update_marque(*args):
    data_pays = data[data['countries_fr'] == dropdown_pays_mq.value]
    dropdown_marque.options = data_pays['brands'].value_counts().index.sort_values()
    dropdown_marque.value = dropdown_marque.options[0]
    graph_nutriscore_marque(dropdown_marque,dropdown_pays_mq)

nutr_marque_output = widgets.Output()
dropdown_pays_mq.observe(update_marque, 'value')
dropdown_marque.observe(graph_nutriscore_marque, 'value')

widgets.interact(graph_nutriscore_marque, Pays = dropdown_pays_mq, Marques = dropdown_marque)
widgets.VBox(
    [
        dropdown_pays_mq,
        dropdown_marque, 
        nutr_marque_output
    ]
)


interactive(children=(Output(),), _dom_classes=('widget-interact',))

In [9]:
# ACP
data_fr = data[data['countries_fr'] == 'France']
data_pca = data_fr[["proteins_100g","salt_100g",\
                    "fat_100g","carbohydrates_100g","fiber_100g",\
                   "additives_n","nb_ingredients",'energy_100g']]

X = data_pca.values
names = data_pca.index
features = data_pca.columns

In [10]:
# centrage et réduction
std_scale = preprocessing.StandardScaler().fit(X)
X_scaled = std_scale.transform(X)

In [11]:
# Calcul des composantes principales
nb_var = 8
pca = decomposition.PCA(n_components=nb_var)
pca.fit(X_scaled)

PCA(n_components=8)

## Projection des produits sur les axes richesse nutritive / transformation industrielle

In [12]:

def graph_ACP(*args):
    if dropdown_categ_1.value == None :
        update_categ(dropdown_pays)
    if dropdown_produit_1.value == None :
        update_prod(dropdown_categ_1)    
    with nutr_marque_output_1:
        nutr_marque_output_1.clear_output(True)   
        data_pays = data[data['countries_fr'] == dropdown_pays_1.value]
        data_pays_categ = data_pays[data_pays['main_category_fr'] == dropdown_categ_1.value]
        data_pays_categ_prod = data_pays_categ[data_pays_categ['product_name'] == dropdown_produit_1.value]
        data_pays_categ_prod_filtre = data_pays_categ_prod[["proteins_100g","salt_100g",\
                                                            "fat_100g","carbohydrates_100g","fiber_100g",\
                                                            "additives_n","nb_ingredients", 'energy_100g']]
        
        X_filtree = data_pays_categ_prod_filtre.values
        names_filtree = data_pays_categ_prod['brands'].values
        features_filtree = data_pays_categ_prod_filtre.columns
        std_scale_filtree = preprocessing.StandardScaler().fit(X_filtree)
        X_scaled_filtree = std_scale.transform(X_filtree)
        X_projected_filtree = pca.transform(X_scaled_filtree)

        display_factorial_planes(X_projected_filtree, nb_var, pca, [(0,1)], labels = np.array(names_filtree),\
                             illustrative_var = np.array(data_pays_categ_prod['product_name'].values))

nutr_marque_output_1 = widgets.Output()

dropdown_pays_1 = widgets.Dropdown(
    value = 'France',
    options = data['countries_fr'].value_counts().index,
    description = 'Liste des pays:',
    style = {'description_width': 'initial'}
    )

dropdown_categ_1 = widgets.Dropdown(
    description = 'Liste des catégories:',
    style = {'description_width': 'initial'}
    )

dropdown_produit_1 = widgets.Dropdown(
    description = 'Liste des produits:',
    style = {'description_width': 'initial'}
    )

display(nutr_marque_output_1, dropdown_pays_1, dropdown_categ_1, dropdown_produit_1)

def update_categ(*args):
    data_pays = data[data['countries_fr'] == dropdown_pays_1.value]
    dropdown_categ_1.options = data_pays['main_category_fr'].value_counts().index.sort_values()
    dropdown_categ_1.value = dropdown_categ_1.options[0]
    update_prod(dropdown_categ_1)
    
def update_prod(*args):
    data_pays = data[data['countries_fr'] == dropdown_pays_1.value]
    data_pays_categ = data_pays[data_pays['main_category_fr'] == dropdown_categ_1.value]
    dropdown_produit_1.options = data_pays_categ['product_name'].value_counts().index
    dropdown_produit_1.value = dropdown_produit_1.options[0]
    
def update_graph(*args):
    graph_ACP(dropdown_produit_1,dropdown_categ_1,dropdown_pays_1)

dropdown_pays_1.observe(update_categ, 'value')
dropdown_categ_1.observe(update_prod, 'value')
dropdown_produit_1.observe(update_graph, 'value')

update_categ(dropdown_pays_1)


Output()

Dropdown(description='Liste des pays:', index=1, options=('États-Unis', 'France', 'Suisse', 'Allemagne', 'Espa…

Dropdown(description='Liste des catégories:', options=(), style=DescriptionStyle(description_width='initial'),…

Dropdown(description='Liste des produits:', options=(), style=DescriptionStyle(description_width='initial'), v…